In [1]:
import glob
import json
import os
from pathlib import Path
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def get_library_size(end_library):
    if end_library is None:
        return 0
    else:
        cnt = 0
        for l in end_library:
            if l is not None:
                cnt += 1
    return cnt
    
def get_expressions_data(dir, globstr, savename=None):
    data = []
    for f in dir.glob(globstr):
        run = int(re.search("run_(\d+)\.", f.name).group(1))
        dataset = re.search("vsize\.([a-zA-Z]+)\.", f.name).group(1)
        d = json.load(f.open("r"))
        results = d["results"]
        for res in results:
            task_name = re.search('name=(.*),', res["task"]).group(1)
            end_library = res["ending_library"]
            end_library_size = get_library_size(end_library)
            data.append((run, task_name, dataset, res["attempt"], res["success"], res["elapsed_time"], res["num_unique_values"], res["num_values_explored"], end_library_size))
        df = pd.DataFrame.from_records(data, columns=["Run", "TaskName", "DataSet", "Attempt", "Success", "ElapsedTime", "NumUniqueValues", "NumValuesExplored", "EndLibrarySize"])
        if savename:
            df.to_pickle(savename)
    return df

In [3]:
%cd "/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results_nodup/"
globstr = "run*.json"
dir = Path("/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results_nodup/")
data = get_expressions_data(dir, globstr)

/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results_nodup


In [4]:
data

,Run,TaskName,DataSet,Attempt,Success,ElapsedTime,NumUniqueValues,NumValuesExplored,EndLibrarySize
0,3,stackoverflow6.sl,sygus,1,True,1.017864,68,207,0
1,4,37534494.sl,sygus,1,True,2.103282,341,1476,0
2,4,addPlusSign,new,1,True,3.072879,66,348,0
3,4,get-middle-name-from-full-name.sl,sygus,1,False,55.881557,3152,50090,1
4,4,get-middle-name-from-full-name.sl,sygus,2,True,12.362254,1867,13850,1
...,...,...,...,...,...,...,...,...,...
949,1,stringLength,new,1,True,2.323558,289,1423,0
950,1,name-combine-2.sl,sygus,1,False,54.837914,3586,50010,0
951,1,exceljet1.sl,sygus,1,True,1.108426,87,311,0
952,2,phone-9.sl,sygus,1,False,50.907802,2984,50022,0


In [76]:
# %cd "~/Envs/augmente_bustle_results/bustle_results_nodup"
# d = pickle.load(Path("sygus-50k_4restart_nogrow-aggregated.pkl").open("rb"))

/home/ken/Envs/augmente_bustle_results/bustle_results_nodup


In [5]:
data

,Run,TaskName,DataSet,Attempt,Success,ElapsedTime,NumUniqueValues,NumValuesExplored,EndLibrarySize
0,3,stackoverflow6.sl,sygus,1,True,1.017864,68,207,0
1,4,37534494.sl,sygus,1,True,2.103282,341,1476,0
2,4,addPlusSign,new,1,True,3.072879,66,348,0
3,4,get-middle-name-from-full-name.sl,sygus,1,False,55.881557,3152,50090,1
4,4,get-middle-name-from-full-name.sl,sygus,2,True,12.362254,1867,13850,1
...,...,...,...,...,...,...,...,...,...
949,1,stringLength,new,1,True,2.323558,289,1423,0
950,1,name-combine-2.sl,sygus,1,False,54.837914,3586,50010,0
951,1,exceljet1.sl,sygus,1,True,1.108426,87,311,0
952,2,phone-9.sl,sygus,1,False,50.907802,2984,50022,0


In [ ]:
import pandas as pd

for dataset in ["sygus", "new"]:
    temp_data = []
    df = data[(data["DataSet"] == dataset) & (data["Success"] == True)]
    for i in range(1, 86502, 100):
        filtered_df = df[df["ElapsedTime"] <= i].groupby("Run").count()
        # Ensure the DataFrame is not empty
        if not filtered_df.empty:
            mean = filtered_df.mean()[0]
            std = filtered_df.std()[0]
            temp_data.append({"seconds": i, "avg": mean, "std": std})
        else:
            temp_data.append({"seconds": i, "avg": 0, "std": 0})  # Or handle this case as you see fit
    
    path = "/Users/habib/Documents/ags-bee-search/scripts/result_processor_crossbeam/stopping_fix/time_data"

    file_name = f'{dataset}-time.csv'
    file_path = os.path.join(path, file_name)
    pd.DataFrame(temp_data).to_csv(file_path, index=False)
                        


In [ ]:
%cd "~/Envs/augmente_bustle_results/bustle_results_nodup"
globstr = "run*.json"
dirs = ["50k_nogrow", "50k_2restart_grow", "50k_2restart_nogrow", "50k_4restart_grow", "50k_4restart_nogrow", "50k_8restart_grow", "50k_8restart_nogrow"]
for dir in dirs:
    df = get_expressions_data(Path(dir), globstr, f"{dir}.pkl")
    df2 = df.drop(columns=["NumUniqueValues", "EndLibrarySize", "ElapsedTime", "Attempt"])
    df2 = df2.groupby(by=["Run", "TaskName"], as_index=False).sum()
    newdf = df2[df2["DataSet"] == "new"].drop(columns=["DataSet"])
    sygusdf = df2[df2["DataSet"] == "sygus"].drop(columns=["DataSet"])
    newdf.to_pickle(f"new-{dir}-aggregated.pkl")
    sygusdf.to_pickle(f"sygus-{dir}-aggregated.pkl")

In [13]:
%cd "/home/ken/Projects/augmented_bustle_results/augmented/bustle_results_nodup/"
d = pickle.load(Path("50k_4restart_nogrow.pkl").open("rb"))
x = d.groupby(by=["TaskName", "Run"]).sum()
x.loc["11604909.sl"]

[Errno 2] No such file or directory: '/home/ken/Projects/augmented_bustle_results/augmented/bustle_results_nodup/'
/home/ken/Envs/augmente_bustle_results/bustle_results_nodup


,DataSet,Attempt,Success,ElapsedTime,NumUniqueValues,NumValuesExplored,EndLibrarySize
Run,,,,,,,
1,sygus,1,1,8.801042,1375,9795,0
2,sygus,1,1,3.593925,774,4201,0
3,sygussygus,3,1,17.017261,2738,19415,0
4,sygus,1,1,10.156737,1526,11396,0
5,sygus,1,1,2.801516,607,2798,0


In [ ]:
d.gr

In [ ]:
def get_tasknames(runname):
    names = set()
    d = json.load(Path(runname).open("r"))
    results = d["results"]
    for res in results:
        task_name = re.search('name=(.*),', res["task"]).group(1)
        names.add(task_name)
    return names
%cd "~/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours/600s"
sygus_tasknames = get_tasknames("run_1.50000.attempt-86400.600.grow-True.vw-bustle_sig-vsize.sygus.json")
new_tasknames = get_tasknames("run_1.50000.attempt-86400.600.grow-True.vw-bustle_sig-vsize.new.json")
print(len(sygus_tasknames))
print(len(new_tasknames))

In [187]:
sygus_task_name_dir = "/Users/habib/Documents/ags-bee-search/config/bustle_benchmarks.txt"
sygus_tasknames = tuple(Path(sygus_task_name_dir).open("r").read().splitlines())
new_task_name_dir = "/Users/habib/Documents/ags-bee-search/config/bustle_38.txt"
new_tasknames = tuple(Path(new_task_name_dir).open("r").read().splitlines())

In [188]:
def get_partial_time_results( globstr):
    solved_data = []
    for f in glob.glob(globstr):
        print(f)
        run = int(re.search("run_(\d+)\.", f).group(1))
        d = json.load(open(f, "r"))
        results = d["results"]
        for i in range(len(results)):
            if results[i]["success"]:
                task_name = re.search('name=(.*),', results[i]["task"]).group(1)
                attempt = results[i]["attempt"]
                attempts_data = results[i - attempt + 1 : i + 1]
                explored = sum(r["num_values_explored"] for r in attempts_data)
                seconds = sum(r["elapsed_time"] for r in attempts_data)
                library_size = sum(get_library_size(r["ending_library"]) for r in attempts_data)
                solved_data.append((run, task_name, attempt, True, library_size, explored, seconds))
    df = pd.DataFrame.from_records(solved_data, columns=["Run", "TaskName", "Attempts", "Success", "EndLibrarySize", "NumValuesExplored", "ElapsedTime"])
    return df

In [189]:
!cd "/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results_nodup/"
print(os.getcwd())
partial_sygus_timedata = get_partial_time_results("run*sygus*.json")
partial_new_timedata = get_partial_time_results("run*new*.json")

/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results_nodup
run_3.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.stackoverflow6.sl.json
run_4.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.37534494.sl.json
run_4.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.get-middle-name-from-full-name.sl.json
run_5.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.phone-3.sl.json
run_5.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.38871714.sl.json
run_2.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.extract-word-containing-specific-text.sl.json
run_4.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.strip-non-numeric-characters.sl.json
run_4.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.33619752.sl.json
run_5.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.sygus.most-frequently-occurring-text.sl.json
run_4.50000.attempt-86400.3600.grow-True.vw-bustle_sig-vsize.

In [171]:
print(partial_sygus_timedata.head(n=5))
print(partial_new_timedata.head(n=5))

   Run                           TaskName  Attempts  Success  EndLibrarySize  \
0    3                  stackoverflow6.sl         1     True               0   
1    4                        37534494.sl         1     True               0   
2    4  get-middle-name-from-full-name.sl         2     True               2   
3    5                         phone-3.sl         1     True               0   
4    5                        38871714.sl         1     True               0   

   NumValuesExplored  ElapsedTime  
0                207     1.017864  
1               1476     2.103282  
2              63940    68.243810  
3               1045     1.636704  
4               2827     3.302809  
   Run                  TaskName  Attempts  Success  EndLibrarySize  \
0    4               addPlusSign         1     True               0   
1    4      padSpaceToGivenWidth         2     True               2   
2    4                 pathDepth         2     True               2   
3    1           su

In [190]:
print(partial_sygus_timedata.shape)
print(partial_new_timedata.shape)

(419, 7)
(189, 7)


In [191]:
def get_partial_task_time_results(globstr):
    data = []
    for f in Path(dir).glob(globstr):
        run = int(re.search("run_(\d+)\.", f.name).group(1))
        d = json.load(open(f, "r"))
        results = d["results"]
        task_name = re.search('name=(.*),', results[0]["task"]).group(1)
        attempts = 0
        explored = 0
        seconds = 0
        library_size = 0
        for i in range(len(results)):
            attempts += 1
            explored += results[i]["num_values_explored"]
            seconds += results[i]["elapsed_time"]
            library_size += get_library_size(results[i]["ending_library"])
            if i == len(results) - 1:
                success = results[i]["success"]
                if d == "86400s" or success:
                    data.append((run, task_name, attempts, success, library_size, explored, seconds))
    df = pd.DataFrame.from_records(data, columns=["Run", "TaskName", "Attempts", "Success", "EndLibrarySize", "NumValuesExplored", "ElapsedTime"])
    return df

In [192]:
# %cd "/home/ken/Projects/augmented_bustle_results/augmented/bustle_results_nodup/50k_24hours/"
# dirs = ["3600s", "21600s", "86400s"]
partial_sygus_tasks_timedata = get_partial_task_time_results("run*sygus*json")
partial_new_tasks_timedata = get_partial_task_time_results("run*new*json")

In [193]:
sygus_timedata = pd.concat([partial_sygus_timedata], ignore_index=True)
new_timedata = pd.concat([partial_new_timedata], ignore_index=True)

In [194]:
sygus_timedata
print(new_timedata.head(n=5))

   Run                  TaskName  Attempts  Success  EndLibrarySize  \
0    4               addPlusSign         1     True               0   
1    4      padSpaceToGivenWidth         2     True               2   
2    4                 pathDepth         2     True               2   
3    1           suffixRemainder         1     True               0   
4    4  containsStringIgnoreCase         2     True               2   

   NumValuesExplored  ElapsedTime  
0                348     3.072879  
1              58176    58.492346  
2              51939    58.045780  
3                835     1.814019  
4              52453    51.822147  


In [195]:
import itertools
def verify_timedata(df, tasknames):
    not_in_df = []
    for run, taskname in itertools.product(range(1,6), tasknames):
        rows = df.loc[(df["TaskName"] == taskname) & (df["Run"] == run)]
        # print(run, taskname, len(rows))
        if len(rows) == 0:
            print(run, taskname, "not in DF")
            not_in_df.append((run, taskname))
        elif len(rows) > 1:
            print(run, taskname, "has multiple entries in DF")
    return not_in_df

In [196]:
# print(sygus_timedata.head(n=5))
# print(len(sygus_tasknames))
# print("SYGUS")
# sygus_manual_entries = verify_timedata(sygus_timedata, sygus_tasknames)
print("NEW")
new_manual_entries = verify_timedata(new_timedata, new_tasknames)

NEW
1 replaceAllCompany not in DF
2 prependMrOrMs has multiple entries in DF
2 replaceAllCompany not in DF
3 prependMrOrMs has multiple entries in DF
3 replaceAllCompany not in DF
4 prependMrOrMs has multiple entries in DF
4 replaceAllCompany not in DF
5 prependMrOrMs has multiple entries in DF
5 replaceAllCompany not in DF


In [179]:
def get_manual_entries(entries):
    data = []
    for run, tname in entries:
        data.append((run, tname, np.nan, False, np.nan, np.nan, np.nan))
    df = pd.DataFrame.from_records(data, columns=["Run", "TaskName", "Attempts", "Success", "EndLibrarySize", "NumValuesExplored", "ElapsedTime"])
    return df

In [180]:
manual_df = get_manual_entries(sygus_manual_entries)
sygus_time_df = pd.concat([sygus_timedata, manual_df], ignore_index=True)
sygus_time_df["Dataset"] = "sygus"
new_timedata["Dataset"] = "new"
time_data = pd.concat([sygus_time_df, new_timedata], ignore_index=True)
time_data.to_pickle("50k_24hours.pkl")

In [239]:
%cd "/home/ken/Projects/augmented_bustle_results/augmented/bustle_results_nodup/50k_24hours"
d = pickle.load(Path("50k_24hours.pkl").open("rb"))
d

/home/ken/Projects/augmented_bustle_results/augmented/bustle_results_nodup/50k_24hours


,Run,TaskName,Attempts,Success,EndLibrarySize,NumValuesExplored,ElapsedTime,Dataset
0,5,bikes.sl,1.0,True,0.0,1067.0,9.816742,sygus
1,5,dr-name.sl,1.0,True,0.0,5123.0,4.048520,sygus
2,5,firstname.sl,1.0,True,0.0,392.0,0.283824,sygus
3,5,initials.sl,1.0,True,0.0,39606.0,40.223683,sygus
4,5,lastname.sl,1.0,True,0.0,330.0,0.252106,sygus
...,...,...,...,...,...,...,...,...
630,3,twoLetterAcronym,1.0,True,0.0,1735.0,1.343204,new
631,3,twoLetterAcronymCapitalization,1.0,True,0.0,45229.0,43.137411,new
632,1,replaceAllCompany,11.0,True,65.0,509895.0,481.067420,new
633,2,replaceAllCompany,17.0,True,9.0,841193.0,775.001053,new
